# 训练过程中将回调函数作用于模型
--------
    * 模型检查点
    * 提前终止
    * 在训练过程中动态调节某些参数值
    * 在训练过程中记录训练指标和验证指标,或将模型学到的表示可视化(这些表示也在不断更新)

## ModelCheckpoint 与 EarlyStopping 回调函数

In [1]:
import keras

callbacks_list = [
    # 如果不在改善,就中断训练
    keras.callbacks.EarlyStopping(
        monitor='acc',  # 监控模型的验证精度
        patience=1,  # 如果精度在多于一轮的时间(即两轮)内不再改善,中断训练
    ),
    # 在每轮过后保存当前权重
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',  # 目标模型文件的保存路径
        monitor='val_loss',
        save_best_only=True  # 如果val_loss没有改善,那么不需要覆盖模型文件.这就可以始终保存在训练过程中见到的最佳模型
    )
]

model.compile(
    optimizer='rsmprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=128,
    callbacks=callbacks_list,
    validation_data=(x_val, y_val)
)

Using TensorFlow backend.


## ReduceLROnPlateau 回调函数

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',  # 监控模型的验证损失
        factor=0.1,  # 触发时将学习率除以10
        patience=10  # 如果验证损失在10轮内没有改善,那么就触发这个回调函数
    )
]

model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=128,
    callbacks=callbacks_list,
    validation_data=(x_val, y_val)
)

## 编写自己的回调函数

In [ ]:
# 在每轮次结束后将模型每层的激活保存到硬盘(格式为Numpy数组),这个激活是对验证集的第一个样本计算得到的
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.modles.Model(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data')
        validation_sample = self.validation_data[0][0:1]    
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_'+str(epoch)+'.npz', 'w')
        np.savez(f, activations)
        f.close()
        

# TensorFlow的可视化框架:TensorBoard

## 使用了TensorBoard的文本分类模型

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


## 为TensorBoard日志文件创建一个目录

In [4]:
%mkdir my_log_dir

In [6]:
word_index_dict = imdb.get_word_index()
reverse_word_index_list = [v for k,v in word_index_dict.items()]
reverse_word_index_list

[34701,
 52006,
 52007,
 16816,
 63951,
 1408,
 16115,
 2345,
 2289,
 52008,
 52009,
 11307,
 40830,
 30568,
 52010,
 40831,
 52011,
 19313,
 52012,
 52013,
 25242,
 6746,
 52014,
 52015,
 52016,
 68804,
 52017,
 40833,
 34702,
 2338,
 40834,
 34703,
 52018,
 16817,
 1636,
 16818,
 52019,
 34704,
 52020,
 11585,
 57766,
 52021,
 14129,
 52023,
 11038,
 52025,
 29114,
 52027,
 52125,
 40835,
 52028,
 52130,
 34706,
 27631,
 40836,
 15492,
 52030,
 11926,
 4010,
 3230,
 52031,
 34707,
 30585,
 52033,
 40837,
 26338,
 52034,
 30569,
 52035,
 52036,
 40839,
 52037,
 52038,
 11927,
 16819,
 52039,
 25243,
 21905,
 52040,
 40840,
 40841,
 359,
 5034,
 7093,
 21906,
 23379,
 52041,
 52042,
 18510,
 30570,
 9878,
 40842,
 52043,
 52044,
 52045,
 40843,
 34708,
 25244,
 7180,
 52046,
 40844,
 11586,
 20598,
 52047,
 11037,
 52048,
 52049,
 52050,
 17633,
 52051,
 30602,
 40846,
 52052,
 52053,
 23380,
 52054,
 27633,
 52055,
 10307,
 52057,
 42577,
 15493,
 40847,
 52058,
 22922,
 52059,
 27634

## 使用一个TensorBoard回调函数来训练模型

In [9]:
import numpy as np
embeddings_layer_names = set(layer.name for layer in model.layers if layer.name.startswith('dense_'))
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir/',  # 日志文件将被写入这个位置
        histogram_freq=1,  # 每一轮之后记录激活直方图
        embeddings_freq=1,  # 每一轮之后记录嵌入数据
    )
]

history = model.fit(
    x_train,
    y_train,
    epochs=20, 
    batch_size=128, 
    validation_split=0.2,
    callbacks=callbacks
)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 58us/step - loss: 0.2465 - acc: 0.9004 - val_loss: 0.3172 - val_acc: 0.8684


ValueError: To visualize embeddings, embeddings_data must be provided.

In [3]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')
plot_model(model, show_shapes=True, to_file='model1.png')